# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [5]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. 
'''

## 2. Generate the mathematical model

In [6]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [7]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Supplier Selection

**Parameters:**

* **D:** Total demand (units): 239,600,480
* **S_i:** Setup cost for vendor i (i = A, B, C, D, E)
    * S_A = $3855.34
    * S_B1 = $125,804.84 (22M - 70M units)
    * S_B2 = $269,304.84 (70M+1 - 100M units)
    * S_B3 = $464,304.84 (100M+1 - 150M units)
    * S_B4 = $761,304.84 (150M+1 - 160M units)
    * S_C = $13,456.00
    * S_D = $6,583.98
    * S_E1 = $0 (0 - 42M units)
    * S_E2 = $84,000 (42M+1 - 77M units) 
* **C_ij:** Unit cost for vendor i within quantity range j (per thousand units)
    * C_A = $61.150
    * C_B1 = $68.099 (22M - 70M units)
    * C_B2 = $66.049 (70M+1 - 100M units)
    * C_B3 = $64.099 (100M+1 - 150M units)
    * C_B4 = $62.119 (150M+1 - 160M units)
    * C_C = $62.019
    * C_D = $72.488
    * C_E1 = $70.150 (0 - 42M units)
    * C_E2_base = $68.150 (42M+1 - 77M units), decreasing by 0.05% per million units. 
* **Cap_i:** Maximum supply capacity for vendor i (units)
    * Cap_A = 33,000,000
    * Cap_B = 160,000,000
    * Cap_C = 165,600,000
    * Cap_D = 12,000,000
    * Cap_E = 77,000,000

**Decision Variables:**

* **x_ij:** Quantity (in thousands of units) purchased from vendor i within quantity range j
    * x_A: quantity purchased from Vendor A
    * x_B1: quantity purchased from Vendor B (22M - 70M)
    * x_B2: quantity purchased from Vendor B (70M+1 - 100M)
    * x_B3: quantity purchased from Vendor B (100M+1 - 150M)
    * x_B4: quantity purchased from Vendor B (150M+1 - 160M)
    * x_C: quantity purchased from Vendor C
    * x_D: quantity purchased from Vendor D
    * x_E1: quantity purchased from Vendor E (0 - 42M)
    * x_E2: quantity purchased from Vendor E (42M+1 - 77M) 

**Objective Function:**

Minimize total cost:

```
Minimize Z =  S_A + C_A * x_A + S_B1 * y_B1 + C_B1 * x_B1 + S_B2 * y_B2 + C_B2 * x_B2 + S_B3 * y_B3 + C_B3 * x_B3 + S_B4 * y_B4 + C_B4 * x_B4 + S_C + C_C * x_C + S_D + C_D * x_D + S_E1 * y_E1 + C_E1 * x_E1 + S_E2 * y_E2 + Σ_(k=1)^35 (C_E2_base - 0.0005 * (k-1)) * x_E2_k 
```

Where:

* y_ij are binary variables, equal to 1 if the corresponding x_ij is greater than 0, and 0 otherwise.
* x_E2_k represents the quantity (in millions of units) purchased within the k-th million units of range E2, capped at 1 million units each.

**Constraints:**

1. **Demand satisfaction:**
    * x_A + x_B1 + x_B2 + x_B3 + x_B4 + x_C + x_D + x_E1 + x_E2 = D / 1000

2. **Capacity constraints:**
    * x_A <= Cap_A / 1000
    * x_B1 + x_B2 + x_B3 + x_B4 <= Cap_B / 1000
    * x_C <= Cap_C / 1000
    * x_D <= Cap_D / 1000
    * x_E1 + x_E2 <= Cap_E / 1000

3. **Vendor B minimum purchase:**
    * x_B1 >= 22,000 * y_B1
    * x_B1 <= 70,000 * y_B1
    * x_B2 >= 70,001 * y_B2
    * x_B2 <= 100,000 * y_B2
    * x_B3 >= 100,001 * y_B3
    * x_B3 <= 150,000 * y_B3
    * x_B4 >= 150,001 * y_B4
    * x_B4 <= 160,000 * y_B4

4. **Vendor E decreasing price:**
    * x_E2 = Σ_(k=1)^35 x_E2_k
    * x_E2_k <= 1000 for all k
 
5. **Non-negativity constraints:**
    * All x_ij >= 0

This model can be implemented in an optimization solver to find the optimal allocation of units from each supplier to minimize the total cost.


## 3. Generate the pyomo code

In [8]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [9]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
D = 239600480  # Total demand in units
S = {"A": 3855.34, 
     "B1": 125804.84, "B2": 269304.84, "B3": 464304.84, "B4": 761304.84,
     "C": 13456.00, 
     "D": 6583.98,
     "E1": 0, "E2": 84000
     }  # Setup costs for each vendor and range
C = {"A": 61.150, 
     "B1": 68.099, "B2": 66.049, "B3": 64.099, "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150 
     }  # Unit costs for each vendor and range
Cap = {"A": 33000000, "B": 160000000, "C": 165600000, "D": 12000000, "E": 77000000}  # Capacity for each vendor

# Create model
model = pyo.ConcreteModel()

# Define sets
vendors = ["A", "B", "C", "D", "E"]
vendor_ranges = {"A": [1], "B": [1, 2, 3, 4], "C": [1], "D": [1], "E": [1, 2]}
E2_ranges = list(range(1, 36))

# Define variables
model.x = pyo.Var(vendors, vendor_ranges, within=pyo.NonNegativeReals)
model.y = pyo.Var(vendors, vendor_ranges, within=pyo.Binary)
model.x_E2_k = pyo.Var(E2_ranges, within=pyo.NonNegativeReals)

# Define objective function
def objective_rule(model):
    cost = 0
    for i in vendors:
        for j in vendor_ranges[i]:
            cost += S[i + str(j)] * model.y[i, j] + C[i + str(j)] * model.x[i, j] 

    cost += sum((C["E2_base"] - 0.0005 * (k - 1)) * model.x_E2_k[k] for k in E2_ranges)  
    return cost

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_rule(model):
    return sum(model.x[i, j] for i in vendors for j in vendor_ranges[i]) == D / 1000

model.demand = pyo.Constraint(rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i, j] for j in vendor_ranges[i]) <= Cap[i] / 1000

model.capacity = pyo.Constraint(vendors, rule=capacity_rule)

def vendor_b_rule(model, j):
    if j == 1:
        return model.x["B", j] >= 22000 * model.y["B", j] 
    elif j == 2:
        return model.x["B", j] >= 70001 * model.y["B", j] 
    elif j == 3:
        return model.x["B", j] >= 100001 * model.y["B", j] 
    elif j == 4:
        return model.x["B", j] >= 150001 * model.y["B", j] 

model.vendor_b_min = pyo.Constraint(vendor_ranges["B"], rule=vendor_b_rule)

def vendor_b_rule2(model, j):
    if j == 1:
        return model.x["B", j] <= 70000 * model.y["B", j]
    elif j == 2:
        return model.x["B", j] <= 100000 * model.y["B", j]
    elif j == 3:
        return model.x["B", j] <= 150000 * model.y["B", j]
    elif j == 4:
        return model.x["B", j] <= 160000 * model.y["B", j]

model.vendor_b_max = pyo.Constraint(vendor_ranges["B"], rule=vendor_b_rule2)

def vendor_e_decreasing_rule(model):
    return model.x["E", 2] == sum(model.x_E2_k[k] for k in E2_ranges)

model.vendor_e_decreasing = pyo.Constraint(rule=vendor_e_decreasing_rule)

def vendor_e_cap_rule(model, k):
    return model.x_E2_k[k] <= 1000

model.vendor_e_cap = pyo.Constraint(E2_ranges, rule=vendor_e_cap_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Print results
print(results)
print("Optimal solution:")
for i in vendors:
    for j in vendor_ranges[i]:
        print(f"x[{i}, {j}]: {model.x[i, j].value}")

for k in E2_ranges:
    print(f"x_E2_{k}: {model.x_E2_k[k].value}")

print(f"Total cost: {model.obj()}")
```

**Explanation:**

1. **Import pyomo:** `import pyomo.environ as pyo` imports the Pyomo library, which is used for optimization modeling.
2. **Sample Data:** This section defines the provided data for demand, setup costs, unit costs, and capacities.
3. **Create a model:** `model = pyo.ConcreteModel()` creates a concrete model object, which will hold the problem formulation.
4. **Define sets:** This section defines sets for vendors and their corresponding quantity ranges to structure the data and variables.
5. **Define variables:**
    - `model.x = pyo.Var(vendors, vendor_ranges, within=pyo.NonNegativeReals)` defines the quantity purchased from each vendor within each quantity range as non-negative real numbers.
    - `model.y = pyo.Var(vendors, vendor_ranges, within=pyo.Binary)` defines binary variables that indicate whether a particular vendor-range combination is used.
    - `model.x_E2_k = pyo.Var(E2_ranges, within=pyo.NonNegativeReals)` defines variables for the quantity purchased within each million units of Vendor E's range E2.
6. **Define objective function:** `model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)` sets up the objective function to minimize the total cost, using the `objective_rule` function to calculate it.
7. **Define constraints:**
    - `model.demand = pyo.Constraint(rule=demand_rule)` ensures that the total purchased quantity meets the total demand.
    - `model.capacity = pyo.Constraint(vendors, rule=capacity_rule)` enforces the capacity constraints for each vendor.
    - `model.vendor_b_min` and `model.vendor_b_max` set the minimum and maximum purchase quantities for each range of Vendor B.
    - `model.vendor_e_decreasing = pyo.Constraint(rule=vendor_e_decreasing_rule)` links the total quantity purchased from Vendor E's range E2 to the individual million-unit quantities.
    - `model.vendor_e_cap = pyo.Constraint(E2_ranges, rule=vendor_e_cap_rule)` limits the maximum quantity purchased within each million units of Vendor E's range E2.
8. **Solve the model:** `solver = pyo.SolverFactory('cbc')` selects the CBC solver, and `results = solver.solve(model)` solves the optimization problem.
9. **Print results:** This section prints the solver's status, the optimal solution (the values of decision variables), and the total cost.

This code provides a Pyomo implementation of the mathematical optimization model for supplier selection. You can run this code using a Python interpreter with Pyomo and CBC solver installed to get the optimal allocation of units from each supplier.


## 4. Input problem data and try running the generated code

In [10]:
import pyomo.environ as pyo

# Sample Data
D = 239600480  # Total demand in units
S = {"A": 3855.34, 
     "B1": 125804.84, "B2": 269304.84, "B3": 464304.84, "B4": 761304.84,
     "C": 13456.00, 
     "D": 6583.98,
     "E1": 0, "E2": 84000
     }  # Setup costs for each vendor and range
C = {"A": 61.150, 
     "B1": 68.099, "B2": 66.049, "B3": 64.099, "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150 
     }  # Unit costs for each vendor and range
Cap = {"A": 33000000, "B": 160000000, "C": 165600000, "D": 12000000, "E": 77000000}  # Capacity for each vendor

# Create model
model = pyo.ConcreteModel()

# Define sets
vendors = ["A", "B", "C", "D", "E"]
vendor_ranges = {"A": [1], "B": [1, 2, 3, 4], "C": [1], "D": [1], "E": [1, 2]}
E2_ranges = list(range(1, 36))

# Define variables
model.x = pyo.Var(vendors, vendor_ranges, within=pyo.NonNegativeReals)
model.y = pyo.Var(vendors, vendor_ranges, within=pyo.Binary)
model.x_E2_k = pyo.Var(E2_ranges, within=pyo.NonNegativeReals)

# Define objective function
def objective_rule(model):
    cost = 0
    for i in vendors:
        for j in vendor_ranges[i]:
            cost += S[i + str(j)] * model.y[i, j] + C[i + str(j)] * model.x[i, j] 

    cost += sum((C["E2_base"] - 0.0005 * (k - 1)) * model.x_E2_k[k] for k in E2_ranges)  
    return cost

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define constraints
def demand_rule(model):
    return sum(model.x[i, j] for i in vendors for j in vendor_ranges[i]) == D / 1000

model.demand = pyo.Constraint(rule=demand_rule)

def capacity_rule(model, i):
    return sum(model.x[i, j] for j in vendor_ranges[i]) <= Cap[i] / 1000

model.capacity = pyo.Constraint(vendors, rule=capacity_rule)

def vendor_b_rule(model, j):
    if j == 1:
        return model.x["B", j] >= 22000 * model.y["B", j] 
    elif j == 2:
        return model.x["B", j] >= 70001 * model.y["B", j] 
    elif j == 3:
        return model.x["B", j] >= 100001 * model.y["B", j] 
    elif j == 4:
        return model.x["B", j] >= 150001 * model.y["B", j] 

model.vendor_b_min = pyo.Constraint(vendor_ranges["B"], rule=vendor_b_rule)

def vendor_b_rule2(model, j):
    if j == 1:
        return model.x["B", j] <= 70000 * model.y["B", j]
    elif j == 2:
        return model.x["B", j] <= 100000 * model.y["B", j]
    elif j == 3:
        return model.x["B", j] <= 150000 * model.y["B", j]
    elif j == 4:
        return model.x["B", j] <= 160000 * model.y["B", j]

model.vendor_b_max = pyo.Constraint(vendor_ranges["B"], rule=vendor_b_rule2)

def vendor_e_decreasing_rule(model):
    return model.x["E", 2] == sum(model.x_E2_k[k] for k in E2_ranges)

model.vendor_e_decreasing = pyo.Constraint(rule=vendor_e_decreasing_rule)

def vendor_e_cap_rule(model, k):
    return model.x_E2_k[k] <= 1000

model.vendor_e_cap = pyo.Constraint(E2_ranges, rule=vendor_e_cap_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')
results = solver.solve(model)

# Print results
print(results)
print("Optimal solution:")
for i in vendors:
    for j in vendor_ranges[i]:
        print(f"x[{i}, {j}]: {model.x[i, j].value}")

for k in E2_ranges:
    print(f"x_E2_{k}: {model.x_E2_k[k].value}")

print(f"Total cost: {model.obj()}")

ERROR: Rule failed when generating expression for Objective obj with index
None: KeyError: 'A1'
ERROR: Constructing component 'obj' from data=None failed:
        KeyError: 'A1'


KeyError: 'A1'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [11]:
print(response.text)

## Mathematical Optimization Model for Supplier Selection

**Parameters:**

* **D:** Total demand (units): 239,600,480
* **S_i:** Setup cost for vendor i (i = A, B, C, D, E)
    * S_A = $3855.34
    * S_B1 = $125,804.84 (22M - 70M units)
    * S_B2 = $269,304.84 (70M+1 - 100M units)
    * S_B3 = $464,304.84 (100M+1 - 150M units)
    * S_B4 = $761,304.84 (150M+1 - 160M units)
    * S_C = $13,456.00
    * S_D = $6,583.98
    * S_E1 = $0 (0 - 42M units)
    * S_E2 = $84,000 (42M+1 - 77M units) 
* **C_ij:** Unit cost for vendor i within quantity range j (per thousand units)
    * C_A = $61.150
    * C_B1 = $68.099 (22M - 70M units)
    * C_B2 = $66.049 (70M+1 - 100M units)
    * C_B3 = $64.099 (100M+1 - 150M units)
    * C_B4 = $62.119 (150M+1 - 160M units)
    * C_C = $62.019
    * C_D = $72.488
    * C_E1 = $70.150 (0 - 42M units)
    * C_E2_base = $68.150 (42M+1 - 77M units), decreasing by 0.05% per million units. 
* **Cap_i:** Maximum supply capacity for vendor i (units)
    * Cap_A = 

In [12]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
D = 239600480  # Total demand in units
S = {"A": 3855.34, 
     "B1": 125804.84, "B2": 269304.84, "B3": 464304.84, "B4": 761304.84,
     "C": 13456.00, 
     "D": 6583.98,
     "E1": 0, "E2": 84000
     }  # Setup costs for each vendor and range
C = {"A": 61.150, 
     "B1": 68.099, "B2": 66.049, "B3": 64.099, "B4": 62.119,
     "C": 62.019,
     "D": 72.488,
     "E1": 70.150,
     "E2_base": 68.150 
     }  # Unit costs for each vendor and range
Cap = {"A": 33000000, "B": 160000000, "C": 165600000, "D": 12000000, "E": 77000000}  # Capacity for each vendor

# Create model
model = pyo.ConcreteModel()

# Define sets
vendors = ["A", "B", "C", "D", "E"]
vendor_ranges = {"A": [1], "B": [1, 2, 3, 4], "C": [1], "D": [1], "E": [1, 2]}
E2_ranges = list(range(1, 36))

# Define variables
model.x = pyo.Var(vendors, vendor_ranges, within=pyo.NonNegativeReals)
model.y = pyo.Var(vendors, vendor_ranges, within=pyo.Binary)
model.x_E2_k = pyo.Var(E2_